In [15]:
import numpy as np
import numpy.random
import holoviews as hv
import matplotlib.pyplot as plt
hv.extension('matplotlib')
hv.extension('bokeh')

In [2]:
%%output backend='matplotlib'
%opts Scatter3D {+framewise}
%opts Overlay {+framewise}

data = np.random.rand(100,3)

def plot_func(elevation, azimuth):
    scatter = hv.Scatter3D(data)
              
    scatter = scatter.opts({'Scatter3D':{'plot': {'fig_size':300, 'azimuth': azimuth, 'elevation':elevation},
                                         'norm': {'framewise':True}}})
    return scatter


holo = hv.HoloMap({(e,a): plot_func(e,a) for e in range(0, 181, 20)
                   for a in range(-90,91,20)}, kdims=['elev','azim'])
holo

In [3]:
%%output backend='matplotlib'

data = np.random.rand(100,3)

def plot_func(elevation, azimuth):
    %%output backend='matplotlib'
    scatter = hv.Scatter3D(data)
              
    scatter = scatter.opts({'Scatter3D':{'plot': { 'azimuth': azimuth, 'elevation':elevation},
                                         'norm': {'framewise':True}}})
    return scatter

scatter = hv.Scatter3D(data)

dmap = hv.DynamicMap(callback=plot_func, kdims=['elevation', 'azimuth'], cache_size=1)
dmap = dmap.redim.range(elevation=(0,180), azimuth=(-90,90)).opts(norm=dict(framewise=True))
dmap

In [4]:
%%output backend='matplotlib'
%opts Scatter3D {+framewise}
%opts Overlay {+framewise}

data = np.random.randint(0,100,[1000,3])

def plot_func(xmax, ymax):
    scatter = hv.Scatter3D(data[(data[:,0] > xmax) & (data[:,1] > ymax)])
    
    return scatter


dmap = hv.DynamicMap(callback=plot_func, kdims=['xmax', 'ymax'], cache_size=1)
dmap = dmap.redim.range(xmax=(0,100), ymax=(0,100)).opts(norm=dict(framewise=True))
dmap

In [5]:
%%output backend='matplotlib'

from ipywidgets import interact, interactive
from bokeh.io import push_notebook, show
from ipykernel.pylab.backend_inline import flush_figures

renderer = hv.renderer('matplotlib')
plot = (renderer.get_plot(scatter))

data2 = np.random.randint(0,100,[1000,3])

scatter2 = hv.Scatter3D(data2)

renderer = hv.renderer('matplotlib')
plot = (renderer.get_plot(scatter2))

def update(a, e):
    %%output backend='matplotlib'

    print(a, e)
    plot.elevation = e
    plot.azimuth = a
    plot.update(0)
    print(plot.elevation)
    display(plot)

interactive_plot = interactive(update, a=(-180,180,1), e=(0,180,1))
output = interactive_plot.children[-1]
output.layout.height='400px'
display(interactive_plot)

In [59]:
%%opts Curve [height=200 width=400 aspect=2]
%%opts ErrorBars [heigh=200 width=400 aspect=2]
%%output backend='matplotlib' size=200

plt.switch_backend('pdf')

arr_a = np.random.rand(10)
arr_b = np.random.rand(10)

nd1 = {}
nd1['one'] = hv.Curve((np.arange(10), arr_a))
nd1['two'] = hv.Curve((np.arange(10)+0.1, arr_b))

nd2 = {}
nd2['one'] = hv.ErrorBars(( np.hstack([np.arange(10)[:,np.newaxis], arr_a[:, np.newaxis], np.random.rand(10,2)])))
nd2['two'] = hv.ErrorBars(( np.hstack([(np.arange(10)+0.1)[:,np.newaxis], arr_b[:, np.newaxis]+0.1, np.random.rand(10,2)])))


ndover1 = hv.NdOverlay(nd1)
ndover2 = hv.NdOverlay(nd2)

ndover1 * ndover2


In [106]:
%%opts Curve {+framewise} [height=200 width=350 aspect=2]
%%opts ErrorBars {+framewise} [height=200 width=350 aspect=2]
%%output backend='bokeh' size=200 holomap='scrubber'

data1 = np.hstack([np.arange(10000)[:,np.newaxis], np.random.rand(10000,3)])
tab1 = hv.Table(data1, kdims='x', vdims=['y','yerr+','yerr-'])
data2 = np.hstack([np.arange(10000)[:,np.newaxis]+0.1, np.random.rand(10000,3)])
tab2 = hv.Table(data2, kdims='x', vdims=['y','yerr+','yerr-'])

def plot(start_time, end_time):
    nd1 = {}
    tab1sel = tab1[0:end_time]
    tab2sel = tab2[0:end_time]
    extents = (start_time, None, end_time, None)
    nd1['one'] = tab1sel.to.curve(kdims=['x'], vdims=['y'], extents=extents)
    nd1['one'].redim(x={'range':(start_time, end_time)})
    nd1['two'] = tab2sel.to.curve(kdims=['x'], vdims=['y'], extents=extents)
    nd1['two'].redim(x={'range':(start_time, end_time)})
    
    nd2 = {}
    nd2['one'] = tab1sel.to.errorbars(kdims=['x'], vdims=['y','yerr+','yerr-'], extents=extents)
    nd2['two'] = tab2sel.to.errorbars(kdims=['x'], vdims=['y','yerr+','yerr-'], extents=extents)


    ndover1 = hv.NdOverlay(nd1)
    ndover1.redim.range(x=(start_time, end_time))
    ndover2 = hv.NdOverlay(nd2)
    ndover2.redim.range(x=(start_time, end_time))
    
    over = ndover1 * ndover2
    over.redim.range(x=(start_time,end_time))
    return over

dmap = hv.DynamicMap(plot, kdims=[hv.Dimension('start_time', range=(0,10000)), hv.Dimension('end_time', range=(0,10000))])

dmap

In [110]:
%%opts Curve {+framewise}
%%output backend='bokeh'

data1 = np.hstack([np.arange(10000)[:,np.newaxis], np.random.rand(10000,3)])
data1 = np.vstack([data1[0:5000], data1[5100:10000]])
tab1 = hv.Table(data1, kdims='x', vdims=['y','yerr+','yerr-'])

def plot(start_time):
    tab1sel = tab1[start_time: start_time+10]
    curv = tab1sel.to.curve(extents=(start_time, None, start_time+10, None))
    
    return curv

dmap = hv.DynamicMap(plot, kdims=[hv.Dimension('start_time', range=(0,10000))])

dmap